In [8]:
import os
import numpy as np
import torch
import pydicom
import matplotlib.pyplot as plt
#from tcia_utils import nbia
from monai.bundle import ConfigParser, download

from monai.transforms import (
    LoadImage,
    Rotate90d,
    LoadImaged,
    Orientation,
    Orientationd,
    EnsureChannelFirst,
    EnsureChannelFirstd,
    Compose,
    SaveImaged
)

from rt_utils import RTStructBuilder
import json

from tqdm import tqdm
settings_rel_path = "settings_lung_segment.json"
#remember this is jupyter notebook, so the path is different

#use os.getcwd() to get the current working directory
settings_path = os.path.join(os.getcwd(), settings_rel_path)
with open(settings_path, 'r') as file:
    settings = json.load(file)


# importances: 0 = normal, 1 = important, 2 = error
def verbose_print(string, importance = 0):
    if importance >= settings["print_treshold"]:
        print(string)
    

verbose_print(settings, importance=0)

input_dir = settings['input_dir']
output_dir = settings['output_dir']


with open(settings['config_path'], 'r') as file:
    config = json.load(file)
inference_path = config["output_dir"]

config = ConfigParser()
config.read_config(settings['config_path'])
preprocessing = config.get_parsed_content("preprocessing")

model = config.get_parsed_content("network")
model.load_state_dict(torch.load(settings["model_path"], map_location=torch.device('cpu')))

inferer = config.get_parsed_content("inferer")
postprocessing = config.get_parsed_content("postprocessing")
verbose_print("Model successfully loaded", importance=0)
#only print if verbose is set to True
#check that input dir exists, if it does not, signal error and exit
#check that output dir exists, if it does not, create it
if not os.path.exists(input_dir):
    verbose_print("ERROR: Input directory does not exist! Terminating program...", importance=2)
    exit(1)
else:
    verbose_print("Input directory found", importance=0)
if not os.path.exists(output_dir):
    verbose_print("Output directory does not exist, creating it...", importance=1)
    os.makedirs(output_dir)
    verbose_print("Output directory created", importance=1)
else:
    verbose_print("Output directory already exists", importance=0)
#list all the files in the input directory
# remove from list all dirs that don't have the prefix settings['prefix']

patients = os.listdir(input_dir)

# let us adjust this with what was written in the previous cell
for patient in tqdm(patients):
    patient_case_study = os.listdir(os.path.join(input_dir, patient))[0]
    patient_cases = os.listdir(os.path.join(input_dir, patient, patient_case_study))
    # remove from patient_cases all files with "Segmenation" in the name
    patient_cases = [x for x in patient_cases if "Segmentation" not in x]
    # extract the first element of the list, which is the first and only file in the directory
    patient_CT = patient_cases[0]
    #obtain the full path of the file
    input_stream = os.path.join(input_dir, patient, patient_case_study, patient_CT)
    output_stream_folder = os.path.join(output_dir, patient)
    
    
    if not os.path.exists(output_stream_folder):
        verbose_print("Output directory does not exist, creating it...", importance=1)
        os.makedirs(output_stream_folder)
        verbose_print("Output directory created", importance=1)
    else:
        verbose_print("Output directory already exists", importance=0)
    
    verbose_print(f"Segmenting {input_stream}", importance=1)

    datalist = [input_stream]
    config['datalist'] = datalist
    dataloader = config.get_parsed_content("dataloader")
    
    #finally we can predict
    X = preprocessing({"image": input_stream})
    with torch.no_grad(): # no backpropagation
        X['pred'] = inferer(X['image'].unsqueeze(0), network = model) # unsqueeze adds a batch dimension
    #this removes the batch dimension in image and pred
    X['pred'] = X['pred'][0]
    X['image'] = X['image'][0]

    X = postprocessing(X)

    X['pred'][X['pred'] < 13] = 0
    X['pred'][X['pred'] > 17] = 0
    image_saver = Compose([SaveImaged(keys = ['pred'], meta_keys = "image_meta_dict", output_dir = output_stream_folder, output_postfix="lung5")])
    image_saver(X)
    verbose_print(f"Five parts segmentation completed", importance=1)
    X['pred'][X['pred'] != 0] = 1
    image_saver = Compose([SaveImaged(keys = ['pred'], meta_keys = "image_meta_dict", output_dir = output_stream_folder, output_postfix="lung")])
    image_saver(X)
    verbose_print(f"Binary segmentation completed", importance=1)
    verbose_print(f"Segmentation completed", importance=1)





    
    
verbose_print("All conversions completed", importance=0)
verbose_print("Program terminated", importance=0)

{'input_dir': 'D:\\nsclc\\tcia\\Rider', 'output_dir': 'D:\\nsclc\\data\\nnUNet_raw\\Dataset002_Rid\\segment_lung', 'model_path': 'C:\\Users\\aless\\Desktop\\git\\NSCLC\\MONAi\\wholeBody_ct_segmentation\\models\\model_lowres.pt', 'config_path': 'C:\\Users\\aless\\Desktop\\git\\NSCLC\\MONAi\\wholeBody_ct_segmentation\\configs\\inference.json', 'prefix': 'R01-', 'num_digits': 3, 'separate_seg': True, 'suffix': 'S', 'print_treshold': 0}
Model successfully loaded
Input directory found
Output directory already exists


  0%|          | 0/1 [00:00<?, ?it/s]

Output directory does not exist, creating it...
Output directory created
Segmenting D:\nsclc\tcia\Rider\RIDER-1129164940\09-20-2006-1-NA-96508\4.000000-NA-24533
2024-03-05 23:29:03,296 INFO image_writer.py:194 - writing: D:\nsclc\data\nnUNet_raw\Dataset002_Rid\segment_lung\RIDER-1129164940\4\4_lung5.nii.gz
Five parts segmentation completed
2024-03-05 23:29:04,766 INFO image_writer.py:194 - writing: D:\nsclc\data\nnUNet_raw\Dataset002_Rid\segment_lung\RIDER-1129164940\4\4_lung.nii.gz


100%|██████████| 1/1 [00:18<00:00, 18.77s/it]

Binary segmentation completed
Segmentation completed
All conversions completed
Program terminated
